In [1]:
import scipy
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import pickle
import math

In [2]:
fDF = pd.read_csv('Data/featureTypes.txt', names=['featureID'])

In [3]:
print fDF.shape
print fDF['featureID'][0]
n = 10000
d = 3000

(3000, 1)
flavors raspberries cherries


In [4]:
trainDF = pd.read_csv('Data/trainData.txt', names = ['instanceID', 'featureID', 'value'], sep=' ')
YDF = pd.read_csv('Data/trainLabels.txt', names = ['label'])
print trainDF.shape[0]

247847


In [5]:
W = np.random.uniform(low=0.0, high=1.0,size = (d,))
B = np.zeros(n)
print W.shape


(3000,)


In [6]:
trainDF[:5]

,instanceID,featureID,value
0,1,13,0.209
1,1,83,0.209
2,1,228,0.209
3,1,242,0.209
4,1,371,0.209


In [7]:
#tDF = csr_matrix(trainDF) 
#print tDF[:5]

In [8]:
#sdf = pd.SparseDataFrame(tDF)
#print sdf[:5]

In [9]:
# Will lead to negative index if re-running
trainDF['instanceID'] -= 1
trainDF['featureID'] -= 1
sMat = csr_matrix((trainDF['value'], (trainDF['featureID'], trainDF['instanceID'])))

In [10]:
#print sMat.shape
#print sMat.todense()

In [11]:
#print sMat[:2]

In [12]:
X = sMat.copy()
#print X

In [13]:
Y = YDF['label'].as_matrix().transpose()
print Y.shape

(10000,)


In [14]:
def initLamda(X, Y):
    YNorm = Y - float(Y.sum())/((float)(Y.shape[0]))
    return 2 * ((X * YNorm).max())
    
print initLamda(X, Y)

896.6381332


In [15]:
#tX = X.copy()
#print tX
#tX.data **= 2
#tA = 2*tX.sum(axis = 1)
#print max(tA), min(tA)
#print tA

In [31]:
# change this to convergence condition

def rmse(input1, input2):
    out = input1 - input2
    #print out
    out **= 2
    out /= len(out)
    error = out.sum()
    return math.sqrt(error)


class Lasso:
    def __init__(self, X, Y, W, B, Lamda):
        self.X = X.copy()
        self.Y = Y.copy()
        self.W = W #.copy()  # Remove this copy later
        self.B = B #.copy()
        t = X.copy()
        t.data **= 2
        self.A = 2*t.sum(axis = 1)
        self.Lamda = Lamda
    
    def loss(self):
        return ((self.X.transpose() * self.W + self.B - self.Y) ** 2).sum() + self.Lamda * (abs(self.W)).sum()
        
    
    def fit(self):
        Lamda = initLamda(self.X, self.Y)
        delta = 0.001
        #print X.shape, W.shape
        #for epoch in range(100):
        oldLoss = self.loss()+2
        newLoss = self.loss()
        while oldLoss - newLoss > delta:
            #print sMat.transpose() * W
            # 4.1.1
            #print t1[:5], t1.shape
            #print t1.shape, B.shape, Y.shape
            XTW = (self.X.transpose() * self.W)
            R = self.Y - (self.X.transpose() * self.W) - self.B
             
            # 4.1.2
            self.B = (R + self.B) / n 
            #self.B = (self.Y - XTW) / n
            #print B.shape
            # 4.1.3
            R = (n-1) * self.B
            #R = self.Y - (XTW + self.B)
            #print R.shape
            #print R[:5]
            # R = R.reshape(-1)
            for ik in range(0, d):
                # 4.1.4
                #ik = 0
                t = (self.X[ik].transpose() * self.W[ik]).toarray().reshape(-1)
                #print t
                #print t.shape
                #print R.shape
                Ck = (2 * self.X[ik] * (R + t)).sum()
                #Ck = (2 * self.X[ik] * (self.Y - self.B - t)).sum()
                #print Ck.sum()
                # Update Weight
                WkOld = self.W[ik]
                if Ck < -Lamda:
                    self.W[ik] = (Ck + Lamda) / self.A[ik]
                elif Ck > Lamda:
                    self.W[ik] = (Ck - Lamda) / self.A[ik]
                else:
                    self.W[ik] = 0
                #print W[ik]
                # 4.1.5
                # print self.W[ik], WkOld
                #print X[ik].toarray().reshape(-1).shape, R.shape
                R = R + self.X[ik].toarray().reshape(-1) * (WkOld - self.W[ik])
                #R = self.Y - (self.X.transpose() * self.W) + self.B
            oldLoss = newLoss
            newLoss = model.loss()
            #print oldLoss, newLoss, oldLoss - newLoss
            print 'LOSS:' , newLoss
            # End of feature vector iterator
    
    def saveModel(self, filename):
        pickle.dump(self, open( filename, "wb" ))
    
    def predict(self, X):
        return (X.transpose() * self.W + self.B)

def loadModel(filename):
    return pickle.load(open(filename, "rb" ))

In [32]:
model = Lasso(X, Y, W, B, initLamda(X.copy(), Y.copy()))
#model.loss()
model.fit()

LOSS: 7601803.37988
LOSS: 7422725.74642
LOSS: 7322749.15688
LOSS: 7259486.81062
LOSS: 7216495.48332
LOSS: 7183507.35544
LOSS: 7156159.39697
LOSS: 7133550.51279
LOSS: 7115190.05367
LOSS: 7101054.28015
LOSS: 7090271.99807
LOSS: 7081809.48935
LOSS: 7075028.09562
LOSS: 7069541.699
LOSS: 7065212.51335
LOSS: 7061904.11812
LOSS: 7059404.70298
LOSS: 7057470.68276
LOSS: 7055913.70742
LOSS: 7054630.83112
LOSS: 7053587.11688
LOSS: 7052751.30498
LOSS: 7052094.94586
LOSS: 7051587.24793
LOSS: 7051197.94786
LOSS: 7050897.21362
LOSS: 7050660.85555
LOSS: 7050472.11483
LOSS: 7050319.31547
LOSS: 7050195.2061
LOSS: 7050095.4037
LOSS: 7050016.66348
LOSS: 7049955.93544
LOSS: 7049909.56969
LOSS: 7049873.9702
LOSS: 7049846.19377
LOSS: 7049824.03107
LOSS: 7049806.04358
LOSS: 7049791.41022
LOSS: 7049779.65167
LOSS: 7049770.36623
LOSS: 7049763.21109
LOSS: 7049757.73558
LOSS: 7049753.51884
LOSS: 7049750.21737
LOSS: 7049747.58174
LOSS: 7049745.45017
LOSS: 7049743.72537
LOSS: 7049742.34585
LOSS: 7049741.2623
LOSS: 

In [33]:
model.saveModel('savedModel')

In [34]:
#dump = pickle.dumps(model)
model2 = loadModel('savedModel')
#model2.fit()
print 'RMSE from loaded model: ', rmse(model2.predict(X), Y)


RMSE from loaded model:  18.3279955561
